In [8]:
import numpy as np
from scipy.optimize import linprog

# Define the data
profit = [10, 20, 15]
machine1 = [2, 4, 3]
machine2 = [3, 2, 4]
max_machine_time = [100, 120]
labor = [3, 2, 4]
max_labor = [100, 80]
storage = [2, 3, 1]
max_storage = [100, 80]
demand = [[50, 40, 60], [60, 50, 40]]

# Number of products
n_products = len(profit)

# Number of machines
n_machines = len(max_machine_time)

# Number of periods
n_periods = len(max_labor)

# Objective function
c = -1 * np.array(profit * n_periods)

# Machine capacity constraints matrix
A_machine = np.zeros((n_machines * n_periods, n_products * n_periods))
for i in range(n_machines):
    for j in range(n_periods):
        A_machine[i * n_periods + j, j * n_products:(j + 1) * n_products] = [machine1[i], machine2[i], 0]

# Machine capacity constraints RHS vector
b_machine = np.array(max_machine_time * n_periods)

# Labor constraints matrix
A_labor = np.zeros((n_periods, n_products * n_periods))
for i in range(n_periods):
    A_labor[i, i * n_products:(i + 1) * n_products] = labor

# Labor constraints RHS vector
b_labor = np.array(max_labor)

# Storage constraints matrix
A_storage = np.zeros((n_periods - 1, n_products * n_periods))
for i in range(n_periods - 1):
    A_storage[i, i * n_products:(i + 2) * n_products] = storage + [-1 * s for s in storage]

# Storage constraints RHS vector
b_storage = np.array(max_storage[:-1])

# Demand constraints matrix
A_demand = np.zeros((n_periods * n_products, n_products * n_periods))
for i in range(n_periods):
    for j in range(n_products):
        A_demand[i * n_products + j, i * n_products + j] = -1

# Demand constraints RHS vector
b_demand = -1 * np.array([d for period in demand for d in period])

# Complete constraints matrix
A = np.vstack([A_machine, A_labor, A_storage, A_demand])

# Complete constraints RHS vector
b = np.hstack([b_machine, b_labor, b_storage, b_demand])

# Solve the linear programming problem
res = linprog(c, A_ub=A, b_ub=b)

# Display the results
if res.success:
    print(f'Optimal production plan: {res.x}')
    print(f'Total profit: {-res.fun}')
else:
    print(f'{res.message}')

The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is At lower/fixed bound)


In [6]:
if res.success:
    # Optimal production plan
    x = res.x.reshape((n_periods, n_products))

    # Machine time allocation
    machine_time = np.zeros((n_machines, n_periods))
    for i in range(n_machines):
        for j in range(n_periods):
            machine_time[i, j] = np.dot([machine1[i], machine2[i], 0], x[j])

    # Display the results
    print(f'Machine time allocation: {machine_time}')
else:
    print(f'{res.message}')


The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is At lower/fixed bound)


تغییرات در پارامترهای در دسترس بودن دستگاه‌ها و نیروی کار می‌تواند بر راه‌حل مسئله تاثیر بگذارد. این تغییرات می‌تواند باعث شود که برنامه تولید بهینه تغییر کند و در نتیجه سود کل نیز تغییر کند.

برای بررسی تاثیر این تغییرات، شما می‌توانید پارامترهای مربوطه را تغییر دهید و سپس مسئله را دوباره حل کنید. به عنوان مثال، فرض کنید که می‌خواهید ظرفیت دستگاه اول در دوره اول را از 100 به 80 کاهش دهید. برای این کار، می‌توانید پارامتر max_machine_time را به صورت زیر تغییر دهید:

In [7]:
# Change the machine capacity
max_machine_time[0] = 80

# Machine capacity constraints RHS vector
b_machine = np.array(max_machine_time * n_periods)

# Complete constraints RHS vector
b = np.hstack([b_machine, b_labor, b_storage, b_demand])

# Solve the linear programming problem again
res = linprog(c, A_ub=A, b_ub=b)

# Display the results
if res.success:
    print(f'New Optimal production plan: {res.x}')
    print(f'New Total profit: {-res.fun}')
else:
    print(f'{res.message}')

The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is At lower/fixed bound)
